In [1]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


In [2]:
import pandas as pd 
from src.database.database import engine
from config.settings import Settings
from src.utils.master_utils import get_osrm_data
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

class MasterManager:
    def __init__(self, configs=None):
        self.config = configs
        if self.config is None:
            self.config = Settings()
            
        self.gps_df = self._load_gps()
        
        self.distance_matrix_path = self.config.distance_matrix_file_path
        self.duration_matrix_path = self.config.duration_matrix_file_path
        
    def _load_gps(self):
        df = pd.read_sql("SELECT * FROM master_gps", engine)
        gps_csv_path = self.config.master_folder_path / self.config.gps_file_name
        df.to_csv(gps_csv_path, index=False)
        return df
    
    # def update_matrix(self):
    #     start = time.time()
    #     if not self.distance_matrix_path.exists():
    #         # gps locations and its shop codes
    #         shop_codes = self.gps_df['shop_code'].values
    #         coords = self.gps_df[['latitude', 'longitude']].values
    #         # create  matrixes
    #         distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         for i in range(len(shop_codes)):
    #             for j  in range(i, len(shop_codes)):
    #                 if i == j:
    #                     distance_matrix[i][j] = 0
    #                     duration_matrix[i][j] = 0
    #                     continue
    #                 origin = tuple(coords[i])
    #                 destination = tuple(coords[j])
                
    #                 _, distance, duration = get_osrm_data(origin, destination)
                    
    #                 distance_matrix[i][j] = round(distance, 3)
    #                 distance_matrix[j][i] = round(distance, 3)
                    
    #                 duration_matrix[i][j] = round(duration, 2)
    #                 duration_matrix[j][i] = round(duration, 2)
    #     print(time.time()-start)
    #     print(distance_matrix)
    #     print(duration_matrix)

    def _get_osrm_thread(self, origin, destination):
        _, distance, duration = get_osrm_data(origin, destination)
        return round(distance, 3), round(duration, 2)

    def update_matrix(self):
        start = time.time()
        if not self.distance_matrix_path.exists():
            # gps locations and its shop codes
            shop_codes = self.gps_df['shop_code'].values
            coords = self.gps_df[['latitude', 'longitude']].values
            # create matrixes
            distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            
            # **THREAD MAGIC: REPLACE YOUR LOOP WITH THESE 6 LINES**
            with ThreadPoolExecutor(max_workers=50) as executor:
                futures = {}
                for i in range(len(shop_codes)):
                    for j in range(i+1, len(shop_codes)):  # Skip i==j & duplicates
                        origin = tuple(coords[i])
                        destination = tuple(coords[j])
                        future = executor.submit(self._get_osrm_thread, origin, destination)
                        futures[future] = (i, j)
                
                for future in as_completed(futures):
                    i, j = futures[future]
                    distance, duration = future.result()
                    distance_matrix[i][j] = distance
                    distance_matrix[j][i] = distance
                    duration_matrix[i][j] = duration
                    duration_matrix[j][i] = duration
                    
            distance_df = pd.DataFrame(distance_matrix, columns=shop_codes, index=list(shop_codes))
            duration_df = pd.DataFrame(duration_matrix, columns=shop_codes, index=list(shop_codes))
            
            distance_df.to_csv(self.distance_matrix_path)
            duration_df.to_csv(self.duration_matrix_path)
            self.gps_df.to_csv(self.config.master_folder_path/'previous_gps.csv', index=False)
        print(time.time() - start)     
        print(distance_matrix)
        print(duration_matrix)

In [16]:
manager = MasterManager()

In [17]:
list(manager.gps_df['shop_code'].values)

['0', '3', '4', '5']

In [18]:
manager.update_matrix()

20.282389640808105
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


In [119]:
manager.update_matrix()

6.137975692749023
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


In [ ]:
from sqlalchemy.orm import Session
from typing import List, Dict, Tuple
from datetime import datetime
from src.utils.master_utils import get_osrm_data
from src.database.models import GPSMaster, MatrixMaster

class DistanceMatrixManager:
    """Manager class for distance matrix operations"""
    
    def __init__(self, db_session: Session):
        self.db = db_session
        
    def calculate_distance(self, lat1: float, lon1: float, 
                          lat2: float, lon2: float) -> float:
        """
        Calculate Haversine distance between two GPS coordinates.
        Returns distance in kilometers.
        """
        _, distance, duration = get_osrm_data((lat1,lon1), (lat2,lon2))
        return distance, duration
    
    def process_pending_updates(self):
        """
        Process all shops marked with matrix_status = 'to_update'.
        Calculate distances and update matrix_status to 'updated'.
        """
        # Get shops that need matrix update
        pending_shops = self.db.query(GPSMaster).filter(
            (GPSMaster.matrix_status == 'to_update') | 
            (GPSMaster.matrix_status == 'to_create')
        ).all()
        
        if not pending_shops:
            print("No pending updates found.")
            return 0
                # Get all shops that are already in the matrix (status = 'updated')
        existing_shops = self.db.query(GPSMaster).filter(
            GPSMaster.matrix_status == 'updated'
        ).all()
        
        total_calculations = 0
        for pending_shop in pending_shops:
            print(f"Processing shop: {pending_shop.shop_code} (ID: {pending_shop.id})")
            
            # Delete old distances if this shop was previously in matrix
            self._delete_shop_distances(pending_shop.id)
            
            # Calculate distances to all existing shops
            distances_added = self._add_shop_to_matrix(
                pending_shop, 
                existing_shops
            )
            
            # Also calculate distances to other pending shops
            for other_pending in pending_shops:
                if other_pending.id != pending_shop.id:
                    self._add_distance_pair(pending_shop, other_pending)
                    distances_added += 1
            
            # Update status to 'updated'
            pending_shop.matrix_status = 'updated'
            total_calculations += distances_added
            

        self.db.commit()
        print(f"\nTotal: Processed {len(pending_shops)} shops, "
              f"{total_calculations} distance calculations")
        
        return total_calculations
    
    def _delete_shop_distances(self, shop_id: int):
        """Delete all distance entries for a specific shop"""
        self.db.query(MatrixMaster).filter(
            (MatrixMaster.shop_id_1 == shop_id) |
            (MatrixMaster.shop_id_2 == shop_id)
        ).delete(synchronize_session=False)
        
        
    def _add_shop_to_matrix(self, new_shop: GPSMaster, 
                           existing_shops: List[GPSMaster]) -> int:
        """
        Add a new shop to the matrix by calculating distances to all existing shops.
        Returns count of distances added.
        """
        
        count = 0
        for existing_shop in existing_shops:
            if existing_shop.id != new_shop.id:
                self._add_distance_pair(new_shop, existing_shop)
                count += 1
        
        return count
    
    def _add_distance_pair(self, shop1: GPSMaster, shop2: GPSMaster):
        """
        Add or update a single distance pair to the matrix.
        Ensures shop_id_1 < shop_id_2 for upper triangle storage.
        """
        distance, time = self.calculate_distance(
            shop1.latitude, shop1.longitude,
            shop2.latitude, shop2.longitude
        )
        
        # Check if distance already exists
        existing = self.db.query(MatrixMaster).filter(
            MatrixMaster.shop_id_1 == shop1.id,
            MatrixMaster.shop_id_2 == shop2.id
        ).first()
        
        if existing:
            existing.distance_km = distance
            existing.time_minutes = time
            existing.last_calculated = datetime.utcnow()
        else:
            new_distance = MatrixMaster(
                shop_id_1=shop1.id,
                shop_id_2=shop2.id,
                shop_code_1=shop1.shop_code,
                shop_code_2=shop2.shop_code,
                distance_km=distance,
                time_minutes=time,
                last_calculated=datetime.utcnow()
            )
            self.db.add(new_distance)

In [7]:
from src.database.database import SessionLocal

manager = DistanceMatrixManager(SessionLocal())

In [8]:
x = manager.process_pending_updates()

Processing shop: 0 (ID: 1)
Processing shop: 3 (ID: 2)
Processing shop: 4 (ID: 3)
Processing shop: 5 (ID: 4)

Total: Processed 4 shops, 12 distance calculations


In [9]:
x

12